In [1]:
import torch
import torch.nn as nn
import os
import time
import math


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/home/peyabi/.conda/envs/ece4420/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/peyabi/.conda/envs/ece4420/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/peyabi/.conda/envs/ece4420/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "/home/peyabi/.conda/envs/ece4420/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in l

In [2]:
# Use CPU explicitly
device = torch.device("cpu")

# Optional: tune number of threads to your machine
torch.set_num_threads(max(1, os.cpu_count() or 1))


def blocked_matmul(A: torch.Tensor, B: torch.Tensor, tile: int = 64) -> torch.Tensor:
    """
    Blocked (tiled) matrix multiplication for CPU using PyTorch tensors.
    A: (N, K)
    B: (K, M)
    tile: tile size (tunable; typical 32..256)
    Returns: C (N, M) computed as A @ B.

    Implementation notes:
    - Uses torch.mm on tile blocks to let CPU BLAS handle inner block multiply.
    - Keeps control over tiling to improve cache locality.
    - Works with float32/float64; ensure tensors are on CPU.
    """
    assert A.device.type == "cpu" and B.device.type == "cpu", "Only CPU tensors supported in this kernel"
    N, K = A.shape
    K2, M = B.shape
    assert K == K2, "Inner dim mismatch"

    # Allocate result
    C = torch.zeros((N, M), dtype=A.dtype, device="cpu")

    # Make contiguous for best performance
    A = A.contiguous()
    B = B.contiguous()

    # Outer loops iterate tiles
    for i in range(0, N, tile):
        i_end = min(i + tile, N)
        A_block = A[i:i_end]               # shape (ti, K)
        for j in range(0, M, tile):
            j_end = min(j + tile, M)
            # We operate on this sub-block of C
            C_block = C[i:i_end, j:j_end]
            # Accumulate across K in tiles
            for p in range(0, K, tile):
                p_end = min(p + tile, K)
                # Multiply small blocks: (ti,k_block) @ (k_block,tj) -> (ti,tj)
                C_block += torch.mm(A_block[:, p:p_end], B[p:p_end, j:j_end])
            # Store accumulator back
            C[i:i_end, j:j_end] = C_block
    return C


# Simple nn.Module wrapper (useful for integrating into model code)
class BlockedMatMulModule(nn.Module):
    def __init__(self, tile: int = 64):
        super().__init__()
        self.tile = tile

    def forward(self, A: torch.Tensor, B: torch.Tensor) -> torch.Tensor:
        return blocked_matmul(A, B, self.tile)


In [3]:
# RAPL ENERGY READING FUNCTIONS
def read_energy_uj(path):
    with open(path, 'r') as f:
        return int(f.read().strip())

def find_rapl_sensors():
    """Return list of RAPL energy sensors (energy_uj files)."""
    base = "/sys/class/powercap/intel-rapl"
    sensors = []
    if not os.path.exists(base):
        print("RAPL not available on this system.")
        return sensors

    for root, dirs, files in os.walk(base):
        for name in files:
            if name == "energy_uj":
                sensors.append(os.path.join(root, name))
    return sensors

RAPL_SENSORS = find_rapl_sensors()

def measure_energy_j(func, *args, **kwargs):
    """Run func and measure elapsed time + energy across all RAPL sensors."""
    if not RAPL_SENSORS:
        raise RuntimeError("No RAPL sensors found!")

    # read initial energy
    e_before = [read_energy_uj(s) for s in RAPL_SENSORS]

    # run function
    t0 = time.perf_counter()
    out = func(*args, **kwargs)
    t1 = time.perf_counter()

    # read final energy
    e_after = [read_energy_uj(s) for s in RAPL_SENSORS]

    # convert to Joules
    e_joules = []
    for before, after, sensor in zip(e_before, e_after, RAPL_SENSORS):
        # handle wrap-around
        if after < before:
            diff = (after + (1 << 32)) - before
        else:
            diff = after - before
        e_joules.append(diff / 1e6)  # µJ → J

    exec_time = t1 - t0
    return out, exec_time, e_joules, RAPL_SENSORS


# FLOP COUNT 
# Energy efficiency is given by GFLOPs per Joule
def matmul_flops(N, K, M):
    # GEMM does 2*N*K*M FLOPs
    return 2 * N * K * M


# BENCHMARK DRIVER
def bench_kernel(name, fn, A, B, N, K, M):
    out, t, sensor_energy, sensors = measure_energy_j(fn, A, B)

    total_energy = sum(sensor_energy)  # J
    avg_power = total_energy / t       # W
    flops = matmul_flops(N, K, M)
    gflops = flops / t / 1e9
    eff = flops / total_energy / 1e9   # GFLOP per Joule

    print(f"\n=== {name} ===")
    print(f"Time: {t*1000:.2f} ms")
    print(f"Total Energy: {total_energy:.4f} J")
    print(f"Average Power: {avg_power:.2f} W")
    print(f"Throughput: {gflops:.2f} GFLOP/s")
    print(f"Energy Efficiency: {eff:.4f} GFLOP/J")

    return out


In [5]:
torch.set_num_threads(8)

N, K, M = 256, 256, 256
A = torch.randn((N, K), dtype=torch.float32)
B = torch.randn((K, M), dtype=torch.float32)

# Baseline
C_ref = bench_kernel("torch.matmul", torch.matmul, A, B, N, K, M)

# Blocked
tile = 64
C_block = bench_kernel(f"blocked_matmul tile={tile}",
                       lambda X, Y: blocked_matmul(X, Y, tile),
                       A, B, N, K, M)
print("Max abs error vs BLAS:", (C_block - C_ref).abs().max().item())

# Naive
def naive(A, B):
    N, K = A.shape
    _, M = B.shape
    C = torch.zeros((N, M), dtype=A.dtype)
    for k in range(K):
        C += A[:, k:k+1].matmul(B[k:k+1, :])
    return C

C_naive = bench_kernel("naive kernel", naive, A, B, N, K, M)
print("Max abs error vs BLAS:", (C_naive - C_ref).abs().max().item())



=== torch.matmul ===
Time: 0.45 ms
Total Energy: 0.1239 J
Average Power: 273.92 W
Throughput: 74.17 GFLOP/s
Energy Efficiency: 0.2708 GFLOP/J

=== blocked_matmul tile=64 ===
Time: 2.09 ms
Total Energy: 0.2297 J
Average Power: 109.73 W
Throughput: 16.03 GFLOP/s
Energy Efficiency: 0.1461 GFLOP/J
Max abs error vs BLAS: 4.1961669921875e-05

=== naive kernel ===
Time: 19.71 ms
Total Energy: 2.3842 J
Average Power: 120.97 W
Throughput: 1.70 GFLOP/s
Energy Efficiency: 0.0141 GFLOP/J
Max abs error vs BLAS: 1.52587890625e-05


In [ ]:
#GPU Version

def gpu_blocked_matmul(A: torch.Tensor, B: torch.Tensor, tile: int = 128) -> torch.Tensor:
    """
    Blocked GPU matmul.
    A: (N, K), B: (K, M)
    tile: tile size for blocking (128 or 256 works well on GPUs)
    """
    assert A.is_cuda and B.is_cuda, "Tensors must be on GPU"

    N, K = A.shape
    K2, M = B.shape
    assert K == K2

    C = torch.zeros((N, M), dtype=A.dtype, device=A.device)

    A = A.contiguous()
    B = B.contiguous()

    for i in range(0, N, tile):
        i_end = min(i + tile, N)
        A_block = A[i:i_end]

        for j in range(0, M, tile):
            j_end = min(j + tile, M)
            C_block = C[i:i_end, j:j_end]

            for p in range(0, K, tile):
                p_end = min(p + tile, K)
                # GPU performs this multiply
                C_block += torch.mm(A_block[:, p:p_end], B[p:p_end, j:j_end])

            C[i:i_end, j:j_end] = C_block

    return C


In [ ]:
class GPUBlockedMatMulModule(nn.Module):
    def __init__(self, tile=128):
        super().__init__()
        self.tile = tile

    def forward(self, A: torch.Tensor, B: torch.Tensor):
        return gpu_blocked_matmul(A, B, self.tile)


In [ ]:
# # move to GPU
# A = torch.randn(1024, 1024, device='cuda')
# B = torch.randn(1024, 1024, device='cuda')

# # baseline
# torch.cuda.synchronize()
# t0 = time.perf_counter()
# C_ref = torch.matmul(A, B)
# torch.cuda.synchronize()
# t1 = time.perf_counter()
# print("torch.matmul:", (t1 - t0)*1000, "ms")

# # blocked
# torch.cuda.synchronize()
# t0 = time.perf_counter()
# C_blk = gpu_blocked_matmul(A, B, tile=128)
# torch.cuda.synchronize()
# t1 = time.perf_counter()
# print("blocked:", (t1 - t0)*1000, "ms")

# # correctness
# print("max error:", (C_ref - C_blk).abs().max().item())


AssertionError: Torch not compiled with CUDA enabled

In [ ]:
import torch
print("torch version:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
print("compiled with cuda:", torch.version.cuda)
print("device count:", torch.cuda.device_count())


SyntaxError: invalid syntax (2505587979.py, line 1)